In [1]:
from google.colab import drive
import numpy as np
import pandas as pd
drive.mount('/content/gdrive')
bengali_news_after_preprocessing = pd.read_pickle('/content/gdrive/My Drive/Projects/Bengali Text Classification/Bengali_Text_after_preprocessing.pkl')
from sklearn.externals import joblib
filename = '/content/gdrive/My Drive/Projects/Bengali Text Classification/fastText_Bangla_content_full.sav'
loaded_model = joblib.load(filename)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
import keras.backend as K
import numpy as np
number_of_sample, max_number_of_words, word_vector_size = 40000, 100, 32
temp = bengali_news_after_preprocessing.loc[:number_of_sample-1,:max_number_of_words-1]

Using TensorFlow backend.


In [0]:
temp = temp.replace(['ঘস', 'ফগ', 'ঝবঃ', 'ঋন', 'ঊঘ', '\u09e4', 'ওৎ', 'গথ', 'খঢ', 'ঝ’', 'ং', 'ঔ', 'ডড', 'গঘ','ঐব','ওঃ’ং','ুী','খষ','ি'], None)
X = np.zeros((number_of_sample, max_number_of_words, word_vector_size), dtype=K.floatx())
for i in temp.index:
  X[i,:,:] = loaded_model.wv[temp.loc[i,:]]

In [0]:
bengali_news = pd.read_pickle('/content/gdrive/My Drive/Projects/Bengali Text Classification/40k_bangla_newspaper_article.p')
bengali_news_dataframe = pd.DataFrame(bengali_news)
y = bengali_news_dataframe['category']

In [0]:
from sklearn import preprocessing
import keras
import numpy as np
le = preprocessing.LabelEncoder()
le.fit(y)
enc = le.transform(y)
y = keras.utils.to_categorical(enc)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, shuffle = True, test_size=0.125)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, shuffle = True, test_size=0.20)

In [7]:
from keras.models import Sequential
from keras.layers import Conv1D, Dropout, Dense, Flatten, LSTM, MaxPooling1D, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, TensorBoard


model = Sequential()

model.add(Bidirectional(LSTM(512, dropout=0.2, recurrent_dropout=0.3), input_shape=(max_number_of_words, word_vector_size)))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.25))

model.add(Dense(13, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

W0614 13:31:34.678698 140338445772672 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0614 13:31:34.692787 140338445772672 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0614 13:31:34.695993 140338445772672 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0614 13:31:36.058312 140338445772672 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0614 13:31:36.072264 

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 1024)              2232320   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
__________

In [9]:
history = model.fit(X_train, y_train, batch_size= 500, shuffle=True, epochs= 100, validation_data=(X_val, y_val))

W0614 13:31:37.277363 140338445772672 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 35000 samples, validate on 4000 samples
Epoch 1/100
35000/35000 [==============================] - 69s 2ms/step - loss: 2.3800 - acc: 0.2004 - val_loss: 1.8559 - val_acc: 0.4445
Epoch 2/100
35000/35000 [==============================] - 66s 2ms/step - loss: 1.7949 - acc: 0.4079 - val_loss: 1.5645 - val_acc: 0.4805
Epoch 3/100
35000/35000 [==============================] - 67s 2ms/step - loss: 1.5341 - acc: 0.5404 - val_loss: 1.3555 - val_acc: 0.6358
Epoch 4/100
35000/35000 [==============================] - 67s 2ms/step - loss: 1.3522 - acc: 0.6162 - val_loss: 1.1977 - val_acc: 0.6650
Epoch 5/100
35000/35000 [==============================] - 67s 2ms/step - loss: 1.2331 - acc: 0.6437 - val_loss: 1.1088 - val_acc: 0.6640
Epoch 6/100
35000/35000 [==============================] - 67s 2ms/step - loss: 1.1473 - acc: 0.6582 - val_loss: 1.0435 - val_acc: 0.6740
Epoch 7/100
35000/35000 [==============================] - 67s 2ms/step - loss: 1.0847 - acc: 0.6699 - val_loss: 0.9874 - v

In [0]:
predicts = model.predict(X_test)
import numpy as np
def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

In [11]:
from sklearn import metrics

print(metrics.accuracy_score(y_test, y_preds))

print(metrics.confusion_matrix(y_test, y_preds))

print(metrics.classification_report(y_test, y_preds))

0.855
[[  4   0   0   0   0   2   0   0   0   2   0   0]
 [  1 252   0   6   3   3   2   0   0  20   1   1]
 [  1   1   0   0   0   1   1   1   0   3   0   0]
 [  0   9   0 101   0   0   0   0   0   9   0   4]
 [  2   2   0   4  12   0   0   0   0   1   0   1]
 [  0   1   0   0   0  62   1   0   0   1   1   0]
 [  0   5   0   0   0   2  42   0   0   8   0   1]
 [  0   2   0   0   1   2   0  28   0   4   0   0]
 [  0   0   0   0   0   0   1   0   0   2   0   0]
 [  1  10   0   0   1   1   1   3   0 237   2   0]
 [  0   2   0   0   1   1   0   0   0   2  73   0]
 [  0   1   0   3   1   0   1   1   0   0   0  44]]
                    precision    recall  f1-score   support

art-and-literature       0.44      0.50      0.47         8
        bangladesh       0.88      0.87      0.88       289
       durporobash       0.00      0.00      0.00         8
           economy       0.89      0.82      0.85       123
         education       0.63      0.55      0.59        22
     entertainment  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
accuracy, val_accuracy = np.array(history.history["acc"]), np.array(history.history["val_acc"])
accuracy, val_accuracy = accuracy.reshape(100,1), val_accuracy.reshape(100,1)
accuracies = np.concatenate((accuracy,val_accuracy),axis=1)
np.savetxt('/content/gdrive/My Drive/Projects/Bengali Text Classification/temp.csv',accuracies,delimiter=",")